## 노트북 내용

이 노트북은 watsonx.ai를 사용한 RAG (Retrieval Augmented Generation)을 시연하기 위한 것입니다. 여기에는 data retrieval, 지식 기본 정보에서 유사도검색 및 모델이 생성한 결과를 확인하는 내용이 포함되어 있습니다.
이를 잘 이해하기 위해서는 Python에 대한 기본 지식이 필요하며 모든 코드는 Python 3.10 으로 작성되어 있습니다.

### RAG (Retrieval Augmented Generation) 
Retrieval Augmented Generation (RAG)는 자연어로 지식 기반 데이터베이스에 질문이나 사실적인 정보를 사용하기를 원하는 다양한 usecase에 활용가능한 패턴입니다.

이 예제는 RAG의 가장 단순한 형태로서 세 가지 단계로 구성되어 있습니다. 

- 텍스트 데이타를 passage로 나누고 embedding하여 지식 기반 데이터베이스 구축
- 지식 기반 데이터베이스로부터 사용자 질문과 가장 유사한 passage들 추출
- 추출된 passage들을 large language model에 입력하여 사용자 질의에 대한 최종 답변 생성.

## 내용

이 노트북은 다음과 같은 단계로 구성되어 있습니다ㅏ.

- [환경설정](#setup)
- [지식기반 데이터베이스 구축](#build_base)
- [chatGPT 모델 접근 설정](#models)
- [사용자 질문에 대한 답변 생성](#predict)

<a id="setup"></a>
## 환경 설정

이노트북에 있는 샘플 코드를 실행하기 전에 다음 작업을 완료해야 합니다.
- 필요한 python package는 conda environment 혹은 python virtual environment에 python 3.10.12 기반의 독립적인 환경을 만든 후 pip install -r requirements_openai.txt를 사용해서 설치.  
- OpenAI의 API를 사용하기 위한 key를 획득해야 한다. 이 key는 <a href="https://platform.openai.com">OpenAI 플래폼</a>에서 설정->사용자 프로파일->사용자 API Keys 에서 생성할 수 있다.

### OpenAI API 사용은 유료이므로 유료 멤버십에 가입해야 정상적으로 실행된다. 그렇지 않은 경우 "현재 사용량이 모두 소진되었다"는 에러가 발생한다.

In [ ]:
!pip install "langchain-community"
!pip install "langchain"
!pip install "sentence-transformers"
!pip install "chromadb"
!pip install "pydantic"
!pip install "tiktoken"
!pip install "openai"

In [ ]:
import os
from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import AutoTokenizer, pipeline
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings

### OpenAI API Key 저장
OpenAI의 API key를 환경변수에 저장한다.

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = 'Your OPEN API Key'

### project id 확인
Foundation model에 접근하기 위한 Cloud Pak for Data의 project id를 확인 한다.
이 노트북이 Foundation model을 제공하는 Cloud Pak for Data이 Project 내에서 실행되는 경우 자동으로 환경 변수에서 가져올 수 있으나 Cloud Pak for Data 외부 환경에서 실행되는 경우는 연관된 project id를 확인한 후에 입력해 줘야 한다.

In [ ]:
try:
    project_id = os.environ["PROJECT_ID"]
    run_in_cp4d = True
except KeyError:
    project_id = "Your Project ID"
    run_in_cp4d = False

<a id="build_base"></a>
## 기본 지식 정보(knowledge base) 구축

The current state-of-the-art in RAG is to create dense vector representations of the knowledge base in order to calculate the semantic similarity to a given user query.
최신 기술인 RAG는 사용자 질의에 대한 sematic similarity를 계산하기 위해 기본 지식 정보에 대한 밀집 벡터 표현 (dense vector represenations)를 생성한다.
이 기본 예제에서는 "영화 분노의 도로-퓨리오사 사가"의 줄거리를 가져와서 chunk로 나누고 embedding 한 후에 Chroma db에 저장하는 것으로 기본 지식 정보를 구축한다.

### Document data 로딩

샘플 코드에서 사용할 데이타 파일은 단순 텍스트 파일이며 git clone으로 생성된 local repository에서 이 노트북이 존재하는 위치의 아래의 data 폴더에 저장되어있다.

In [ ]:
if run_in_cp4d:

    from ibm_watson_studio_lib import access_project_or_space
    wslib = access_project_or_space()

    # load data of type "text/plain" into a file like object
    # raw_data_0 = wslib.load_data('매드맥스-퓨리오사-사가.txt')
    wslib.download_file('퓨리오사-줄거리.txt')
    filename = '퓨리오사-줄거리.txt'
else:
    filename = os.path.join(os.getcwd(), 'data', '퓨리오사-줄거리.txt')

In [ ]:
loader = TextLoader(filename, encoding='UTF8')
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

### embedding function 생성

여기서는 OpenAI에서 제공하는 `text-embedding-ada-002`를 사용한다.
다른 embedding 함수를 사용할 수도 있으나 이 경우 embedding 함수와 사용하고자 하는 vector db (여기서는 Chroma)의 embedding size가 일치해야 한다.
참고로 Chroma db의 embedding size는 768 이다.

In [ ]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

### Vector DB (Chroma db) 생성

chunk로 나누어진 데이터와 embeddng 모델을 사용하여 Chroma db를 생성한다. 이 함수가 실행될 때 각 chunk들이 embedding되고 vector화된 데이터가 원본 텍스트와 함께 db내에 저장된다.

In [ ]:
docsearch = Chroma.from_documents(texts, embeddings)

### 사용자 질의 검색 테스트

앞서 구축한 지식 기반 정보로부터 사용자의 질의와 가장 유사한 정보를 검색해서 출력해 본다.

In [ ]:
retriever = docsearch.as_retriever(search_kwargs={'k': 2})
resutls = retriever.get_relevant_documents("퓨리오사가 한 쪽 팔을 잃게되는 경위가 뭔가요?")
resutls

<a id="models"></a>
## OpenAI의 chatGPT 3.5 Turbo에 대한 접근 설정을 설정한다.

In [ ]:
llm = ChatOpenAI(model = "gpt-3.5-turbo")

<a id="predict"></a>
## RAG를 사용하여 사용자 질문에 대한 답변을 생성한다.

langchain의 `RetrievalQA` (question answering chain) 를 사용하여 RAG 작업을 자동으로 수행한다.

In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

In [ ]:
qa.invoke("퓨리오사가 한 쪽 팔을 잃게되는 경위가 뭔가요?")